# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
weather_data = "./WeatherPy.csv"
vacation_df = pd.read_csv(weather_data)
vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,45.27,77,0,2.19,US,1666357101
1,Hobart,-42.8794,147.3294,63.05,87,75,1.14,AU,1666356965
2,Ushuaia,-54.8000,-68.3000,44.26,57,75,9.22,AR,1666357259
3,Rikitea,-23.1203,-134.9692,70.97,75,8,21.39,PF,1666357260
4,Castro,-24.7911,-50.0119,62.24,97,100,11.25,BR,1666357230


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
#Use the Lat and Lng as locations and Humidity as the weight.
#store lat / long data
locations = vacation_df[["Lat", "Lng"]]
humidity = vacation_df["Humidity"].astype(float)

In [31]:
#create a heat map by location
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Create bank symbol layer
top_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
)

fig = gmaps.figure()
fig.add_layer(bank_layer)

fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
#Drop rows that have null values
new_vacation_data = vacation_df.dropna()
#set the temerature range of the ideal location
clean_vacation_data = new_vacation_data[(new_vacation_data['Max Temp'] >= 70) 
                                        & (new_vacation_data['Max Temp'] <= 80)]
clean_vacation_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,Rikitea,-23.1203,-134.9692,70.97,75,8,21.39,PF,1666357260
5,Marsh Harbour,26.5412,-77.0636,77.14,75,14,4.12,BS,1666357200
9,Hilo,19.7297,-155.0900,77.29,86,13,3.00,US,1666357262
13,São João da Barra,-21.6403,-41.0511,78.93,69,46,18.16,BR,1666357265
15,Reconquista,-29.1500,-59.6500,73.72,55,0,10.18,AR,1666357266
...,...,...,...,...,...,...,...,...,...
542,Cho Dok,10.7000,105.1167,79.21,85,89,6.20,VN,1666357521
547,Lata,40.1629,-8.3327,70.07,73,100,9.66,PT,1666357523
554,Camabatela,-8.1881,15.3750,77.61,59,100,0.98,AO,1666357526
562,La Flèche,47.6982,-0.0755,70.05,54,100,15.86,FR,1666357530


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
hotel_df = pd.DataFrame()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
